# Dogs vs Cats

In [1]:
# Basic imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2

# preprocessing and split
from sklearn.model_selection import train_test_split as split

# models
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegressionCV

# Scores and reports
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, plot_roc_curve
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Make the data frame
def Makedf(file_names):
    li = []
    la = []
    labels = ["cat", "dog"]
    for path in file_names:
        im = cv2.cvtColor(cv2.imread(path), cv2.IMREAD_COLOR)/255
        im = cv2.resize(im,(64,64), interpolation=cv2.INTER_CUBIC)
    
        im = pd.Series(im.flatten())
        for j,label in enumerate(labels):
                if label in path[60:].lower():
                    la.append(j)
        li.append(im)


    frame = pd.concat(li, axis=1, ignore_index=True).T
    frame = frame.rename(columns=lambda s: 'pix ' + str(s), index=lambda s: 'img ' +str(s) )
    frame["Label"] = la

    return frame

In [3]:
# Make the data frame
Train_file = glob.glob(r'/Users/ortrabelsi/Desktop/מטלה דימות נתונים/סופי 1/dogs-vs-cats/train' + "/*")
Test_file = glob.glob(r'/Users/ortrabelsi/Desktop/מטלה דימות נתונים/סופי 1/dogs-vs-cats/test1'+ "/*")
frame = Makedf(Train_file)
frame

,pix 0,pix 1,pix 2,pix 3,pix 4,pix 5,pix 6,pix 7,pix 8,pix 9,...,pix 12279,pix 12280,pix 12281,pix 12282,pix 12283,pix 12284,pix 12285,pix 12286,pix 12287,Label
img 0,0.136327,0.070568,0.030537,0.290097,0.191504,0.127606,0.178991,0.093003,0.069753,0.325438,...,0.347351,0.216840,0.155961,0.333011,0.238576,0.176154,0.311092,0.214389,0.142827,1
img 1,0.597560,0.440309,0.159370,0.643314,0.490059,0.155005,0.638715,0.500511,0.263284,0.659308,...,0.684137,0.506347,0.171911,0.251853,0.133743,-0.001771,0.375202,0.232263,0.026814,0
img 2,0.255425,0.220420,0.378275,0.650781,0.647662,0.652087,0.747219,0.745615,0.653914,0.762110,...,0.492607,0.316669,0.263236,0.434572,0.264806,0.209426,0.422459,0.268242,0.210787,1
img 3,0.057787,0.093081,0.132297,0.052654,0.087948,0.127164,0.060935,0.100295,0.127456,0.066849,...,0.147523,0.193880,0.263014,0.117008,0.170190,0.220688,0.098293,0.157116,0.192523,0
img 4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
img 24995,0.050821,0.093958,0.109644,0.051751,0.092212,0.120909,0.053918,0.106487,0.138654,0.060003,...,0.119339,0.272855,0.420252,0.097660,0.242758,0.376091,0.140537,0.285635,0.418968,1
img 24996,0.082698,0.088031,0.086915,0.365892,0.354685,0.326928,0.364984,0.345492,0.322808,0.091036,...,0.259218,0.314212,0.404248,0.220218,0.274939,0.385195,0.356017,0.425847,0.494387,1
img 24997,0.015056,0.018977,0.011134,0.011765,0.015686,0.007843,-0.000012,0.014066,0.006223,0.008233,...,0.606181,0.699508,0.710227,0.209240,0.295977,0.313033,0.295923,0.364028,0.379244,1
img 24998,0.540009,0.536087,0.481185,0.576028,0.572106,0.517204,0.558261,0.550417,0.507280,0.586935,...,0.003922,0.003922,0.003922,0.003922,0.003922,0.003922,0.003922,0.003922,0.003922,0


In [4]:
# Split Data
y = frame['Label']
X = frame.drop('Label', axis=1)

#split x and y to trin and test
x_train , x_test , y_train , y_test = split(X,y ,train_size = 0.9 , shuffle = True , random_state = 1)


In [5]:
gx = XGBClassifier(n_estimators= 25, n_jobs=-1, learning_rate=0.4, seed=0)
gx.fit(x_train,y_train)

[18:00:26] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.4, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=25, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [9]:
gx.score(x_test,y_test)

0.6624